# **Parallélisation du jeux de la vie**

In [ ]:
#pip install mpi4py

In [ ]:
%%writefile jeux_delavie.py

from mpi4py import MPI
import numpy as np
import sys


#### Une fonction pour afficher la grille avec des symboles
def affichage(grille, length1, length2):
    print("---------------------------------")
    for i in range(0, length1):
        for j in range(0, length2):
            if grille[i,j] == True:
                print('*',end='')
            else:
                print(' ',end='')
        print()
    print("---------------------------------")


#### Proposition pour initialiser la grille avec un motif de type vaisseau
def vaisseau (length1, length2):
    grille = np.zeros((length1, length2),dtype='bool')
    if length1 > 4 and length2 > 6:
        grille[1, 5] = True
        grille[2, 6] = True
        grille[3, 2] = True
        grille[3, 6] = True
        grille[4, 3] = True
        grille[4, 4] = True
        grille[4, 5] = True
        grille[4, 6] = True
    return grille


#### Une petite fonction pour aider la mise à jour de l'état d'une cellule
#### on donne l'état et le nombre de voisins vivants (paramètre n)
#### si la cellule est morte elle revit si le nbre de voisins vivants est 3
#### si la cellule est vivante elle meurt si le nbre de voisins vivants est 2 ou 3.
def vivant_mort(etat, n):
    nouvel_etat = False
    if (etat==False and n==3):
        nouvel_etat = True
    if (etat==True and n<4 and n>1):
        nouvel_etat = True
    return nouvel_etat


#### Une fonction pour calculer le nouvel état de la cellule i, j de grille.
#### Grille non fermé
def etat_cellule(grille, i, j, length1, length2):
    n = 0
    if grille[i, (j+1)%length2]:
      n+=1
    if grille[i-1, (j+1)%length2]:
      n+=1
    if grille[i+1, (j+1)%length2]:
      n+=1
    if grille[i, (j-1+length2)%length2]:
      n+=1
    if grille[i-1, (j-1+length2)%length2]:
      n+=1
    if grille[i+1, (j-1+length2)%length2]:
      n+=1
    if grille[i-1, j]:
      n+=1
    if grille[i+1, j]:
      n+=1
    return vivant_mort(grille[i,j], n)


#### Pour calculer une itération où toutes les cellules sont mises à jour
def iteration(grille, res, length1, length2):
    for i in range(1, length1-1):
        for j in range(0, length2):
            res[i, j] = etat_cellule(grille, i, j, length1, length2)


#### Implementation de la parallelisation

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

length1 = int(sys.argv[1]) #nombre de lignes de la grille
length2 = int(sys.argv[2]) #nombre de colonnes
nb_iter = int(sys.argv[3]) # nombre d'itérations du jeu de la vie
root = int(sys.argv[4])

grille = None
if rank == root:
    grille = vaisseau(length1, length2)

# Decoupage de la grille

n_local = length1 // size
r = length1 % size

count = np.array([(n_local + (1 if i < r else 0)) * length2 for i in range(size)])


if rank < r:
    n_local += 1

grille_local = np.zeros((n_local,length2), dtype='bool')

comm.Scatterv([grille, count, MPI.BOOL], grille_local, root)

# Ghosting

u = np.zeros((n_local + 2, length2), dtype='bool')
res_u = np.zeros_like(u)

u[1:n_local+1, :] = grille_local

# definition des voisins
left = rank - 1 if rank > 0 else None
right = rank + 1 if rank < size - 1 else None

# Boucle principale

for it in range(nb_iter):

    if left is not None:
      comm.Isend(u[1,:], dest=left)
      comm.Irecv(u[0,:], source=left)

    if right is not None:
      comm.Isend(u[n_local,:], dest=right)
      comm.Irecv(u[n_local+1,:], source=right)

    iteration(u, res_u, n_local+2, length2)

    u, res_u = res_u, u   # échange des grilles

# Gathering final

grille_final = None
if rank == root:
    grille_final = np.zeros((length1, length2), dtype='bool')

comm.Gatherv(u[1:n_local+1,:], [grille_final, count, MPI.BOOL], root)

if rank == root:
    affichage(grille_final, length1, length2)


Overwriting jeux_delavie.py


In [ ]:
!OMPI_ALLOW_RUN_AS_ROOT=1 OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 mpirun --oversubscribe -np 4 python3 jeux_delavie1.py 19 15 5 0

---------------------------------
               
               
   *** **      
       **      
               
      **       
               
               
               
               
               
               
               
               
               
               
               
               
               
---------------------------------
